In [2]:
import tensorflow as tf
import argparse
import pandas as pd
import numpy as np

In [3]:
import os 
import random 

In [4]:
parser = argparse.ArgumentParser() 
parser.add_argument("--dataroot", default="../trumptweets-data/")
parser.add_argument("--cuda", action="store_true", help="Enable CUDA/GPU")
parser.add_argument("--batchSize", type=int, default=64) 
parser.add_argument("--cache", default="../trumptweets-data/")
parser.add_argument("--randomSeed") 
options, _ = parser.parse_known_args() 

In [5]:
try: 
    os.makedirs(options.cache) 
except OSError: 
    pass # already exists 

In [6]:
if options.randomSeed is None: 
    options.randomSeed = random.randint(1, 10000) 

tf.random.set_seed(options.randomSeed) 


In [7]:
options.dataroot = "/floyd/input/trumptweets_data/"
options.cache = "../"


In [8]:
# Load CSV dataset import os
import json 
dataset_root = options.dataroot

files = [] 
for file in os.listdir(dataset_root): 
    if file[-4:] == "json": 
        files.append(file) 
objs = [] 
for f in files: 
    with open(dataset_root + f, 'r') as file: 
        objs += json.load(file)




In [9]:
df =pd.DataFrame(objs)

In [10]:
chars = set() 

def add_to_set(text): 
    chars.update(text)
df.text.apply(add_to_set); 
char2idx = {u:i for i, u in enumerate(chars)}
char2idx[None] = len(char2idx.keys())
idx2char = np.array(list(char2idx.keys()))

In [11]:
sequence_length = max(df.text.apply(len))

In [12]:
def convert_to_ints(text): 
    l = [char2idx[c] for c in text] 
    l += [char2idx[None] for i in range( sequence_length - len(text) )]
    return np.array(l) 
numerics = df.text.apply(convert_to_ints)

In [13]:
ds = tf.data.Dataset.from_tensor_slices(numerics.to_list())

In [14]:
# map each sequence 
def split_inputs(chunk): 
    return chunk[:-1], chunk[1:]
dataset = ds.map(split_inputs)

In [27]:
BATCH_SIZE = 512
BUFFER_SIZE = 10000

In [28]:
def get_batch(): 
    return dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [29]:
batch = get_batch()

In [30]:
# build model in this case 

In [31]:
vocab_size = idx2char.shape[0]

In [32]:
embedding_dimensions = vocab_size  
rnn_units = 512

In [33]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size): 
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]), 
        tf.keras.layers.GRU(rnn_units, 
                           return_sequences=True, 
                           stateful=True, 
                           recurrent_initializer='glorot_uniform'), 
        tf.keras.layers.Dense(vocab_size)
    ])
    return model 

In [34]:
model = build_model(vocab_size=vocab_size, 
                    embedding_dim=embedding_dimensions, 
                    rnn_units=rnn_units, 
                    batch_size=BATCH_SIZE)

In [35]:
def loss(labels, logits): 
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True) 

In [24]:
example_loss = loss(target_example_batch, example_predictions)

NameError: name 'target_example_batch' is not defined

In [36]:
model.compile(optimizer='adam', loss=loss)

In [26]:
checkpoint_prefix = os.path.join(options.cache, "ckpt_{epoch}")
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir=options.cache))

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, 
    save_weights_only= True
)

In [37]:
EPOCHS = 15
history = model.fit(batch, epochs = EPOCHS, callbacks = [checkpoint_callback])

Epoch 1/15
10/10 [==============================] - 9s 901ms/step - loss: 4.5012
Epoch 2/15
10/10 [==============================] - 9s 900ms/step - loss: 2.2568
Epoch 3/15
10/10 [==============================] - 9s 906ms/step - loss: 1.7525
Epoch 4/15
10/10 [==============================] - 9s 908ms/step - loss: 1.5573
Epoch 5/15
10/10 [==============================] - 9s 910ms/step - loss: 1.4766
Epoch 6/15
10/10 [==============================] - 9s 916ms/step - loss: 1.4424
Epoch 7/15
10/10 [==============================] - 9s 909ms/step - loss: 1.4192
Epoch 8/15
10/10 [==============================] - 9s 916ms/step - loss: 1.3957
Epoch 9/15
10/10 [==============================] - 9s 921ms/step - loss: 1.3711
Epoch 10/15
10/10 [==============================] - 9s 920ms/step - loss: 1.3296
Epoch 11/15
10/10 [==============================] - 9s 923ms/step - loss: 1.2945
Epoch 12/15
10/10 [==============================] - 9s 920ms/step - loss: 1.2599
Epoch 13/15
10/10 [======

In [20]:
model = build_model(vocab_size, embedding_dim=embedding_dimensions, 
                   rnn_units=rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir=options.cache))
model.build(tf.TensorShape([1, None]))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 256)            83456     
_________________________________________________________________
gru (GRU)                    (1, None, 256)            394752    
_________________________________________________________________
dense (Dense)                (1, None, 326)            83782     
Total params: 561,990
Trainable params: 561,990
Non-trainable params: 0
_________________________________________________________________


In [22]:
def generate_text(model, start_string): 
    # generate characters 
    num_generate = sequence_length 
    
    input_eval = [char2idx[s] for s in start_string] 
    input_eval = tf.expand_dims(input_eval, 0) 
    
    text_generated = [] 
    
    temperature = 0.4
    model.reset_states() 
    
    for i in range(num_generate): 
        predictions = model(input_eval) 
        predictions = tf.squeeze(predictions, 0) 
        
        predictions = predictions / temperature 
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1, 0].numpy() 
        
        input_eval = tf.expand_dims([predicted_id], 0)
        char = idx2char[predicted_id - 1]
        if char == None: 
            char = "" 
        text_generated.append(char)
        
    return (start_string + ''.join(text_generated)) 


In [ ]:

generate_text(model, start_string='')

In [266]:
from collections import Counter
counter = Counter() 
for k in numerics: 
    counter.update(k) 
    